# Covid-19 Dashboard

This is a Covid-19 dashboard that displays data about the amount of hospitalised Covid patients in England and which of those patients are on ventilators alongside the overall death rate. 

You can interact with the graph by highlighting the key with CTRL-Click and also view it logarithmically.

All data has been extract from the UK Governmant and Public Health England

In [20]:
#all necessary imports 
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [21]:
#filters and structures as per PHE dev kit
filters = [
    'areaType=nation',
    'areaName=England'
]

structure = {
    "date": "date",
    "ventilated": "covidOccupiedMVBeds",
    "hospitalised": "hospitalCases",
    "deaths": "cumDeaths28DaysByDeathDateRate"
}

In [23]:
myapi = Cov19API(filters=filters, structure=structure)

In [24]:
casesvbeds=myapi.get_json()

In [25]:
with open("casesvbeds.json", "wt") as OUTF:
    json.dump(casesvbeds, OUTF)

In [26]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [27]:
with open("casesvbeds.json", "rt") as INFILE:
    data=json.load(INFILE)

In [28]:
#creates dataframe
datalist=data['data']

dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

In [32]:
#fills in dataframe
index=pd.date_range(startdate, enddate, freq='D')
casesvbedsf=pd.DataFrame(index=index, columns=['hospitalised', 'ventilated','deaths'])

for entry in datalist: 
    date=parse_date(entry['date'])
    for column in ['hospitalised', 'ventilated','deaths']:
        if pd.isna(casesvbedsf.loc[date, column]):  
            value= float(entry[column]) if entry[column]!=None else 0.0
            casesvbedsf.loc[date, column]=value
            
casesvbedsf.fillna(0.0, inplace=True)

In [37]:
casesvbedsf.to_pickle("casesvbeds.pkl")

In [38]:
#imports time data
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")

In [39]:
#creates refresh button
output=wdg.Output()

@output.capture(clear_output=True, wait=True)
def myapi(button):
    global df
    df=wrangle_data(myapi)
    refresh_graph()
    
    myapi_refresh = datetime.datetime.now()
    apibutton.icon="fa-refresh"
    

    
def myapi(button):
    print("Last refreshed", current_time)
    
apibutton=wdg.Button(
    description='Refresh Data',
    disabled=False,
    button_style='',
    tooltip='Click to download current Public Health England data',
    icon='fa-refresh'
)
   
display(apibutton)
apibutton.on_click(myapi) 
output

Button(description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Click to download current Pub…

Output()

Last refreshed 15:58:02


In [40]:
#add control widgets to graph
series=wdg.SelectMultiple(
    options=['ventilated', 'hospitalised', 'deaths'],
    value=['ventilated', 'hospitalised', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],description='Scale:',
    disabled=False
)

controls=wdg.HBox([series, scale])

def casesvbeds_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        casesvbedsf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
graph=wdg.interactive_output(casesvbeds_graph, {'gcols': series, 'gscale': scale})
ctrls=wdg.VBox([series, scale])
form=wdg.HBox([graph, ctrls])

display(form)



In [30]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
casesvbeds={}

**Author and Copyright Notice** Created by Richard De La Cruz (2020) *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*